In [ ]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from my_functions import z_NB
import numpy as np

In [ ]:
with open('npy/selection.npy', 'rb') as f:
    selection = pickle.load(f)
selection.keys()

In [ ]:
selection['puri'].shape

In [ ]:
# Make DataFrame
df = pd.DataFrame(data=selection)

# Drop some columns
for col in ['src', 'x_im', 'y_im']:
    df = df.drop(col, axis=1)

# Convert tile_id into tile names
tile_dict = {
    2241: 'AEGIS001',
    2243: 'AEGIS002',
    2406: 'AEGIS003',
    2470: 'AEGIS004',
    2520: 'J-NEP'
}

def tile_id_to_name(id):
    return tile_dict[int(id)]

def dec_to_sex(alpha):
    alpha = alpha * 60 * 60
    m, s = divmod(alpha, 60)
    d, m = divmod(m, 60)
    return f'{d:0.0f}º{m:0.0f}``{s:0.1f}\''

def redshift(nb):
    return f'{z_NB(nb)[0]:0.2f}'

def substitute_nans_zspec(what):
    if np.isnan(what):
        return '-'
    else:
        return f'{np.abs(what):0.2f}'

def substitute_nans_spcl(what):
    if what == 'nan':
        return '-'
    else:
        return what

def format_puri(p):
    return f'{p:0.2f}'

df['tile_id'] = df['tile_id'].astype(int).apply(tile_id_to_name)
df['RA'] = df['RA'].apply(dec_to_sex)
df['DEC'] = df['DEC'].apply(dec_to_sex)
df['nb_sel'] = df['nb_sel'].apply(redshift)
df['SDSS_spCl'] = df['SDSS_spCl'].astype(str).apply(substitute_nans_spcl)
df['SDSS_zspec'] = df['SDSS_zspec'].apply(substitute_nans_zspec)
df['puri'] = df['puri'].apply(format_puri)

ew_np = df['EW_lya'].to_numpy()
ew_err_np = df['EW_lya_err'].to_numpy()
ew_str = np.zeros(ew_np.shape).astype(str)
for i, (ew, err) in enumerate(zip(ew_np, ew_err_np)):
    ew_str[i] = f'{ew:0.0f}$\pm${err:0.0f}'
df['EW_lya'] = ew_str
df = df.drop('EW_lya_err', axis=1)

L_np = df['L_lya'].to_numpy()
L_err_np = df['L_lya_err'].to_numpy()
L_str = np.zeros(L_np.shape).astype(str)
for i, (L, err) in enumerate(zip(L_np, L_err_np)):
    L_str[i] = f'{L:0.2f}$\pm${err:0.2f}'
df['L_lya'] = L_str
df = df.drop('L_lya_err', axis=1)

mag = df['r']
df = df.drop('r', axis=1)

# Sort by tile_id
df = df.sort_values(['tile_id', 'L_lya'])
df.head()

In [ ]:
df.to_csv('csv/selection_table.csv')

In [ ]:
L_lya = np.array([L_text[:5] for L_text in df['L_lya']]).astype(float)
zspec = np.array([0 if item == '-' else float(item) for item in df['SDSS_zspec']])
z_Arr = df['nb_sel'].to_numpy().astype(float)
EW_Arr = ew_np
puri = df['puri'].to_numpy().astype(float)
df.keys()

In [ ]:
xmsdss = pd.read_csv('csv/xmatch_sdss_dr12.csv', sep=',', header=1)
zph = xmsdss['zph'].to_numpy()
e_zph = xmsdss['e_zph'].to_numpy()
xmsdss

In [ ]:
from my_functions import count_true
sdss_mask = (df['SDSS_spCl'] != '-') 
puri_mask = (puri > 0)
L_mask = (L_lya > 44)
cont_mask = sdss_mask & L_mask & puri_mask & (np.abs(zspec - z_Arr) > 0.3)
conf_mask = sdss_mask & L_mask & puri_mask & ~(np.abs(zspec - z_Arr) > 0.3)
N_cont = count_true(cont_mask)
N_confirmed = count_true(conf_mask)

perc = N_confirmed / (N_confirmed + N_cont)
perc_e = ((N_confirmed + N_cont)**-4 * (N_cont**2 * N_confirmed + N_confirmed**2 * N_cont))**0.5
print(f'Confirmed sources: {N_confirmed} / {(N_confirmed + N_cont)} ; p = {perc:0.2f}({perc_e:0.2f})')

avgpuri_total = np.mean(puri[L_mask])
avgpuri_sdss = np.mean(puri[sdss_mask & L_mask])
print(f'Average Total purity = {avgpuri_total:0.3f}')
print(f'Average SDSS purity = {avgpuri_sdss:0.3f}')

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))

ax.scatter(mag[conf_mask], L_lya[conf_mask], color='green')
ax.scatter(mag[cont_mask], L_lya[cont_mask], color='red')

# ax.set_yscale('log')
# ax.set_ylim(0, 200)
ax.set_xlabel('r')
ax.set_ylabel('L_lya')

plt.show()

In [ ]:
def do_this():
        fig, ax = plt.subplots(figsize=(6, 3.5))

        bins = np.linspace(1, 4, 15 + 1)
        bins_c = [bins[i: i + 2].sum() * 0.5 for i in range(len(bins) - 1)]
        bins_sep = bins_c[1] - bins_c[0]
        EW_mock_good_h = np.load(f'npy/Expected_EW_hist.npy')
        EW_mock_cont_h = np.load(f'npy/Cont_EW_hist.npy')

        EW_this_h = np.histogram(np.log10(EW_Arr), bins=bins)[0] / count_true(EW_Arr)

        ax.plot(bins_c + bins_sep * 0.5, EW_this_h, drawstyle='steps', color='dimgray',
                zorder=99, label='miniJPAS + J-NEP')

        ax.plot(bins_c, EW_mock_good_h + EW_mock_cont_h, color='orange', ls='--',
                label='Mock (full selection)')
        ax.plot(bins_c, EW_mock_good_h, color='red', ls=':', label='Mock (only LAEs)')

        ax.set_ylabel('Fraction of sources', fontsize=12)
        ax.set_xlabel(r'log EW$_0^{\mathrm{Ly}\alpha}$ ($\AA$)', fontsize=12)

        ax.legend(fontsize=12)

        fig.savefig(f'figures/EW_hist.pdf', bbox_inches='tight',
                facecolor='w', edgecolor='w')
        plt.show()

do_this()